In [74]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

In [75]:
# load data
pcd_file = "/home/nived/logs/digiforest_analysis_test_data/anymal_data/payload_clouds_no_ground/cloud_1683207437_293473482.pcd"
pcd = o3d.io.read_point_cloud(pcd_file)
print(pcd)
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.5,
                                  front=[0.79, 0.02, 0.60],
                                  lookat=[2.61, 2.04, 1.53],
                                  up=[-0.60, -0.012, 0.79])

PointCloud with 181285 points.


In [ ]:
# DBSCAN (open3d)
eps = 0.5
min_cluster_size = 100
labels = np.array(pcd.cluster_dbscan(eps=eps, min_points=min_cluster_size, print_progress=False))
max_label = labels.max()
print("num_clusters = %d" % (max_label + 1))

In [ ]:
# DBSCAN (sklearn)
from sklearn.cluster import DBSCAN
eps = 0.5
min_cluste_size = 100
db = DBSCAN(eps=0.3, min_samples=min_cluster_size).fit(X)
labels = db.labels_
max_label = labels.max()
print("num_clusters = %d" % (max_label + 1))

In [ ]:
# HDBSCAN
from sklearn.cluster import HDBSCAN
X = np.asarray(pcd.points)
min_cluster_size = 100
hdb = HDBSCAN(min_cluster_size=20)
hdb.fit(X)
labels = hdb.labels_
max_label = labels.max()
print("num_clusters = %d" % (max_label + 1))

In [ ]:
# OPTICS (O(n^2)- Not Suitable for large pointclouds)
from sklearn.cluster import OPTICS, cluster_optics_dbscan

optics = OPTICS(min_samples=50, xi=0.05, max_eps=1.0, algorithm="kd_tree")
optics.fit(X)
labels = cluster_optics_dbscan(reachability=optics.reachability_,
                               core_distances=optics.core_distances_,
                               ordering=optics.ordering_,
                               eps=0.5)
max_label = labels.max()
print("num_clusters = %d" % (max_label + 1))

In [78]:
# KMeans
from sklearn.cluster import KMeans
num_clusters = 100

X = np.asarray(pcd.points)
labels = KMeans(n_clusters=num_clusters, n_init= "auto").fit_predict(X)
max_label = labels.max()
print("num_clusters = %d" % (max_label + 1))

num_clusters = 50


In [83]:
# Bisecting KMeans
from sklearn.cluster import BisectingKMeans
num_clusters = 100

X = np.asarray(pcd.points)
labels = BisectingKMeans(n_clusters=num_clusters).fit_predict(X)
max_label = labels.max()
print("num_clusters = %d" % (max_label + 1))

num_clusters = 100


In [90]:
# Euclidean clustering
from sklearn.neighbors import KDTree
X = np.asarray(pcd.points)

ec = EuclideanClustering(X, distance_threshold = 1.0, min_cluster_size=100, max_cluster_size=10000)


In [87]:
# Visualize segmentation results
cmap = plt.get_cmap("tab20")
colors = cmap(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.5,
                                  front=[0.79, 0.02, 0.60],
                                  lookat=[2.61, 2.04, 1.53],
                                  up=[-0.60, -0.012, 0.79])

In [89]:
print(max_labels)

25


In [91]:
X_tree

In [ ]:
X_tree.